# Load libraries

In [122]:
# Supress Warnings 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import fiona


# Import common GIS tools
import numpy as np

import matplotlib.pyplot as plt
import rasterio.mask
from matplotlib.cm import jet,RdYlGn

# Import Planetary Computer tools
import stackstac
import pystac_client
import planetary_computer 
from odc.stac import stac_load

# Not sure that I would use
import folium


# Multi-dimensional arrays and datasets
import xarray as xr

# Geospatial raster data handling
import rioxarray as rxr

# Geospatial data analysis
import geopandas as gpd

# Geospatial operations
import rasterio as rio
from rasterio import windows  
from rasterio import features  
from rasterio import warp
from rasterio.warp import transform_bounds 
from rasterio.windows import from_bounds 



# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Planetary Computer Tools
import pystac_client
import planetary_computer as pc
from pystac.extensions.eo import EOExtension as eo

# Others
import os
from tqdm import tqdm
from rtree import index

In [123]:
import osmnx as ox
from shapely.geometry import Point, Polygon


# Load training data


In [124]:
ground_df = pd.read_csv("Training_data_uhi_index_2025-02-18.csv")
print(ground_df.shape)
ground_df.head()

(11229, 4)


,Longitude,Latitude,datetime,UHI Index
0,-73.909167,40.813107,24-07-2021 15:53,1.030289
1,-73.909187,40.813045,24-07-2021 15:53,1.030289
2,-73.909215,40.812978,24-07-2021 15:53,1.023798
3,-73.909242,40.812908,24-07-2021 15:53,1.023798
4,-73.909257,40.812845,24-07-2021 15:53,1.021634


In [125]:
#Date time convert
ground_df['datetime'] = pd.to_datetime(ground_df['datetime'])


In [126]:
place_name = "New York City, USA"
counties = ox.features_from_place(place_name, tags={"boundary": "administrative", "admin_level": "7"})

borough_names = ["Manhattan", "The Bronx"]
counties = counties[counties['name'].isin(borough_names)]


ground_df['geometry'] = ground_df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
gdf_points = gpd.GeoDataFrame(ground_df, geometry='geometry', crs=counties.crs)

if "index_right" in gdf_points.columns:
    gdf_points.drop(columns=["index_right"], inplace=True)
# Join
ground_df= gpd.sjoin(gdf_points,counties, how='left', predicate='within')

# Longitude/Latitude/datetime/UHI Index
ground_df = ground_df[['Longitude','Latitude','datetime','UHI Index','geometry','name']]
ground_df.rename(columns={'name':"Counties"}, inplace=True)

In [127]:
ground_df


,Longitude,Latitude,datetime,UHI Index,geometry,Counties
0,-73.909167,40.813107,2021-07-24 15:53:00,1.030289,POINT (-73.90917 40.81311),The Bronx
1,-73.909187,40.813045,2021-07-24 15:53:00,1.030289,POINT (-73.90919 40.81304),The Bronx
2,-73.909215,40.812978,2021-07-24 15:53:00,1.023798,POINT (-73.90922 40.81298),The Bronx
3,-73.909242,40.812908,2021-07-24 15:53:00,1.023798,POINT (-73.90924 40.81291),The Bronx
4,-73.909257,40.812845,2021-07-24 15:53:00,1.021634,POINT (-73.90926 40.81284),The Bronx
...,...,...,...,...,...,...
11224,-73.957050,40.790333,2021-07-24 15:57:00,0.972470,POINT (-73.95705 40.79033),Manhattan
11225,-73.957063,40.790308,2021-07-24 15:57:00,0.972470,POINT (-73.95706 40.79031),Manhattan
11226,-73.957093,40.790270,2021-07-24 15:57:00,0.981124,POINT (-73.95709 40.79027),Manhattan
11227,-73.957112,40.790253,2021-07-24 15:59:00,0.981245,POINT (-73.95711 40.79025),Manhattan


In [128]:
ground_df["Counties"].value_counts()

Counties
Manhattan    5819
The Bronx    5410
Name: count, dtype: int64

# Weather data

- 1- Mapping the weather for Manhattan and Bronx from additional data

### Bronx and Manhattan

In [129]:
dataset_bronx = pd.read_excel("Additional dataset/NY_Mesonet_Weather.xlsx", sheet_name= "Bronx")
dataset_manhattan = pd.read_excel("Additional dataset/NY_Mesonet_Weather.xlsx", sheet_name="Manhattan")

In [130]:

#change datetime of weather bronx and weather manhattan
dataset_bronx['Date / Time'] = pd.to_datetime(dataset_bronx['Date / Time'])
dataset_manhattan['Date / Time'] = pd.to_datetime(dataset_manhattan['Date / Time'])
dataset_bronx = dataset_bronx.sort_values('Date / Time')
dataset_manhattan = dataset_manhattan.sort_values('Date / Time')

In [131]:
dataset_bronx

,Date / Time,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,2021-07-24 06:00:00,19.3,88.2,0.8,335,12
1,2021-07-24 06:05:00,19.4,87.9,0.8,329,18
2,2021-07-24 06:10:00,19.3,87.6,0.7,321,25
3,2021-07-24 06:15:00,19.4,87.4,0.5,307,33
4,2021-07-24 06:20:00,19.4,87.0,0.2,301,42
...,...,...,...,...,...,...
164,2021-07-24 19:40:00,24.9,49.0,3.5,184,24
165,2021-07-24 19:45:00,24.8,49.0,3.3,173,19
166,2021-07-24 19:50:00,24.9,48.7,3.8,168,17
167,2021-07-24 19:55:00,24.9,47.3,4.1,171,16


#### Merge data

In [132]:
# Sort dataframes by datetime for asof merge
ground_df = ground_df.sort_values('datetime')

# Merge Bronx data
bronx_data = pd.merge_asof(
    ground_df[ground_df['Counties'] == 'The Bronx'],
    dataset_bronx,
    left_on='datetime',
    right_on='Date / Time',
    direction='nearest'  # Matches nearest timestamp
)

# Merge Manhattan data
manhattan_data = pd.merge_asof(
    ground_df[ground_df['Counties'] == 'Manhattan'],
    dataset_manhattan,
    left_on='datetime',
    right_on='Date / Time',
    direction='nearest'
)

# Combine the two datasets back into one
final_df = pd.concat([bronx_data, manhattan_data])

# Drop redundant 'Date / Time' column if necessary
final_df.drop(columns=['Date / Time'], inplace=True)

# Reset index
final_df.reset_index(drop=True, inplace=True)

print(len(final_df))
final_df

11229


,Longitude,Latitude,datetime,UHI Index,geometry,Counties,Air Temp at Surface [degC],Relative Humidity [percent],Avg Wind Speed [m/s],Wind Direction [degrees],Solar Flux [W/m^2]
0,-73.910167,40.803775,2021-07-24 15:01:00,1.000718,POINT (-73.91017 40.80378),The Bronx,28.0,40.3,3.0,75,725
1,-73.910268,40.803817,2021-07-24 15:01:00,1.000718,POINT (-73.91027 40.80382),The Bronx,28.0,40.3,3.0,75,725
2,-73.910065,40.803733,2021-07-24 15:01:00,0.998554,POINT (-73.91006 40.80373),The Bronx,28.0,40.3,3.0,75,725
3,-73.910367,40.803858,2021-07-24 15:01:00,1.007209,POINT (-73.91037 40.80386),The Bronx,28.0,40.3,3.0,75,725
4,-73.910465,40.803903,2021-07-24 15:01:00,1.007209,POINT (-73.91046 40.8039),The Bronx,28.0,40.3,3.0,75,725
...,...,...,...,...,...,...,...,...,...,...,...
11224,-73.979088,40.771655,2021-07-24 15:59:00,0.975234,POINT (-73.97909 40.77166),Manhattan,27.0,46.1,2.7,209,620
11225,-73.979060,40.771682,2021-07-24 15:59:00,0.975234,POINT (-73.97906 40.77168),Manhattan,27.0,46.1,2.7,209,620
11226,-73.979027,40.771695,2021-07-24 15:59:00,0.975234,POINT (-73.97903 40.7717),Manhattan,27.0,46.1,2.7,209,620
11227,-73.978947,40.771705,2021-07-24 15:59:00,0.975234,POINT (-73.97895 40.7717),Manhattan,27.0,46.1,2.7,209,620


# Footprint data

In [133]:
buildings = gpd.read_file('Additional dataset/Building_Footprint.kml', driver='KML')
buildings

,Name,Description,geometry
0,,,"MULTIPOLYGON (((-73.91903 40.8482, -73.91933 4..."
1,,,"MULTIPOLYGON (((-73.92195 40.84963, -73.92191 ..."
2,,,"MULTIPOLYGON (((-73.9205 40.85011, -73.92045 4..."
3,,,"MULTIPOLYGON (((-73.92056 40.8514, -73.92053 4..."
4,,,"MULTIPOLYGON (((-73.91234 40.85218, -73.91247 ..."
...,...,...,...
9431,,,"MULTIPOLYGON (((-73.95267 40.77923, -73.95254 ..."
9432,,,"MULTIPOLYGON (((-73.94964 40.77613, -73.94931 ..."
9433,,,"MULTIPOLYGON (((-73.9521 40.7688, -73.95174 40..."
9434,,,"MULTIPOLYGON (((-73.9523 40.75904, -73.95246 4..."


In [134]:
buildings = buildings.explode(column="geometry",index_parts=False)
buildings

,Name,Description,geometry
0,,,"POLYGON ((-73.91903 40.8482, -73.91933 40.8479..."
1,,,"POLYGON ((-73.92195 40.84963, -73.92191 40.849..."
2,,,"POLYGON ((-73.9205 40.85011, -73.92045 40.8501..."
3,,,"POLYGON ((-73.92056 40.8514, -73.92053 40.8514..."
4,,,"POLYGON ((-73.91234 40.85218, -73.91247 40.852..."
...,...,...,...
9431,,,"POLYGON ((-73.95267 40.77923, -73.95254 40.779..."
9432,,,"POLYGON ((-73.94964 40.77613, -73.94931 40.776..."
9433,,,"POLYGON ((-73.9521 40.7688, -73.95174 40.76931..."
9434,,,"POLYGON ((-73.9523 40.75904, -73.95246 40.7590..."


In [135]:
final_df["geometry"] = final_df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)
training_gdf = gpd.GeoDataFrame(final_df, geometry="geometry", crs="EPSG:4326")

training_gdf = training_gdf.to_crs(epsg=32618)  # Web Mercator
buildings = buildings.to_crs(epsg=32618)

## FP: Number of buildings & total building area

In [136]:
# Create 100m buffer for each training point
buffer_distance = 700
training_gdf['buffer'] = training_gdf.geometry.buffer(buffer_distance)

In [137]:
# # Initialize building count column
# final_df["building_count_700m"] = 0  
# final_df["building_area_700m"] = 0.0
# training_gdf = training_gdf.reset_index(drop=True)

# # Compute building areas
# final_df["area"] = buildings.geometry.area

# # Loop through each point and count intersecting buildings
# for idx, row in training_gdf.iterrows():
#     buffer_geom = row["buffer"]
#     count = buildings[buildings.geometry.intersects(buffer_geom)].shape[0]
#     training_gdf.at[idx, "building_count_700m"] = count

#     inside_buildings = buildings[buildings.geometry.within(buffer_geom)]
#     # Sum the area of the selected buildings
#     total_area = inside_buildings["area"].sum()
#     # Store the total area in the DataFrame
#     training_gdf.at[idx, "building_area_700m"] = total_area


# Reset index to ensure unique labels for safe assignment
training_gdf = training_gdf.reset_index(drop=True)
final_df = final_df.reset_index(drop=True)

# Initialize columns in training_gdf (not final_df initially)
training_gdf["building_count_700m"] = 0  
training_gdf["building_area_700m"] = 0.0

# Pre-compute building areas
buildings["area"] = buildings.geometry.area

# Loop through each point and calculate count and area
for idx, row in training_gdf.iterrows():
    buffer_geom = row["buffer"]
    
    # Count intersecting buildings
    count = buildings[buildings.geometry.intersects(buffer_geom)].shape[0]
    training_gdf.at[idx, "building_count_700m"] = count

    # Sum area of buildings fully within the buffer
    inside_buildings = buildings[buildings.geometry.within(buffer_geom)]
    total_area = inside_buildings["area"].sum()
    training_gdf.at[idx, "building_area_700m"] = total_area

# Safely assign the results to final_df
final_df["building_count_700m"] = training_gdf["building_count_700m"].astype(int).reset_index(drop=True)
final_df["building_area_700m"] = training_gdf["building_area_700m"].reset_index(drop=True)




In [138]:
final_df['building_area_700m'].describe()

count     11229.000000
mean     374959.745593
std      122411.127712
min       70855.476588
25%      281961.552344
50%      367525.232985
75%      441362.050566
max      790286.857908
Name: building_area_700m, dtype: float64

## FP: Shape compactness

In [139]:
# Calculate shape compactness (P^2 / A)
buildings["perimeter"] = buildings.geometry.length
buildings['area']=buildings.geometry.area
buildings["compactness"] = (buildings["perimeter"] ** 2) / buildings['area']

# Display the updated DataFrame
buildings[["perimeter", "area", "compactness"]].head()




,perimeter,area,compactness
0,122.943986,619.739128,24.389655
1,42.215413,95.264377,18.707319
2,53.448033,141.262523,20.222577
3,41.054764,79.661011,21.158326
4,58.808530,216.099170,16.003963


In [140]:
final_df['mean_compactness_700m']=0.0
for idx, buffer_poly in training_gdf['buffer'].items():
    intersecting_buildings = buildings[buildings.geometry.intersects(buffer_poly)]
    # Calculate mean compactness of the intersecting buildings
    if not intersecting_buildings.empty:
        mean_compactness = intersecting_buildings["compactness"].mean()
    else:
        mean_compactness = 0  # or np.nan if you prefer
    
    # Store the result in final_df
    final_df.at[idx, "mean_compactness_700m"] = mean_compactness

    

## FP: Mean height of building

In [141]:
ny_buildings = gpd.read_file('./Building Footprints_20250311.geojson')
ny_buildings.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1082803 entries, 0 to 1082802
Data columns (total 16 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   name        2238 non-null     object        
 1   base_bbl    1082803 non-null  object        
 2   shape_area  1082803 non-null  object        
 3   heightroof  1082803 non-null  object        
 4   mpluto_bbl  1082803 non-null  object        
 5   cnstrct_yr  1072590 non-null  object        
 6   globalid    1082803 non-null  object        
 7   lststatype  1082459 non-null  object        
 8   feat_code   1082803 non-null  object        
 9   groundelev  1082254 non-null  object        
 10  geomsource  1082490 non-null  object        
 11  bin         1082803 non-null  object        
 12  lstmoddate  1082803 non-null  datetime64[ms]
 13  doitt_id    1082803 non-null  object        
 14  shape_len   1082803 non-null  object        
 15  geometry    1082803 non-

In [142]:

#convert BIN to str
#buildings['bin'] = buildings['bin'].astype(str)
#subset to only manhattan and bronx through bin starts with either a 1 or 2
nyc_buildings = ny_buildings[ny_buildings['bin'].str.startswith(('1', '2'))]
nyc_buildings.head(2)

,name,base_bbl,shape_area,heightroof,mpluto_bbl,cnstrct_yr,globalid,lststatype,feat_code,groundelev,geomsource,bin,lstmoddate,doitt_id,shape_len,geometry
18,None,1021210037,0.0,59.722628,1021210037,1910,{A0E56BCC-A86B-4CEF-9A42-9B4ECD61743F},Constructed,2100,154,Photogramm,1062896,2017-08-22,708881,0.0,"MULTIPOLYGON (((-73.9387 40.83782, -73.93863 4..."
48,None,2027810500,0.0,10.54654743,2027810500,1973,{2323D1C1-3086-4286-A469-4D0CE8D0756C},Constructed,2100,11,Photogramm,2117853,2014-07-16,813985,0.0,"MULTIPOLYGON (((-73.87287 40.80269, -73.8729 4..."


In [143]:
nyc_buildings.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 149591 entries, 18 to 1082183
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   name        1476 non-null    object        
 1   base_bbl    149591 non-null  object        
 2   shape_area  149591 non-null  object        
 3   heightroof  149591 non-null  object        
 4   mpluto_bbl  149591 non-null  object        
 5   cnstrct_yr  147089 non-null  object        
 6   globalid    149591 non-null  object        
 7   lststatype  149507 non-null  object        
 8   feat_code   149591 non-null  object        
 9   groundelev  149455 non-null  object        
 10  geomsource  149534 non-null  object        
 11  bin         149591 non-null  object        
 12  lstmoddate  149591 non-null  datetime64[ms]
 13  doitt_id    149591 non-null  object        
 14  shape_len   149591 non-null  object        
 15  geometry    149591 non-null  geometry      
dt

In [144]:
nyc_buildings.columns

Index(['name', 'base_bbl', 'shape_area', 'heightroof', 'mpluto_bbl',
       'cnstrct_yr', 'globalid', 'lststatype', 'feat_code', 'groundelev',
       'geomsource', 'bin', 'lstmoddate', 'doitt_id', 'shape_len', 'geometry'],
      dtype='object')

In [145]:
#Drop columns in the geodataframe nyc_buildings
nyc_buildings.drop(columns=['name', 'base_bbl', 'mpluto_bbl',
       'cnstrct_yr', 'globalid', 'lststatype',
        'lstmoddate', 'doitt_id', 'geomsource', 'shape_area', 'shape_len'], inplace=True)

In [146]:
#add a centroid column from geom
nyc_buildings['centroid'] = nyc_buildings['geometry'].centroid

In [147]:

#turn height roof to numberic
ny_buildings['heightroof'] = pd.to_numeric(ny_buildings['heightroof'], errors='coerce')
#convert unit from feet to meter for column heightroof
nyc_buildings['heightroof'] = ny_buildings['heightroof'] * 0.3048 # Code c My cho nay lanyc_buildings['heightroof'] * 0.3048

In [148]:
nyc_buildings.head()

,heightroof,feat_code,groundelev,bin,geometry,centroid
18,18.203457,2100,154,1062896,"MULTIPOLYGON (((-73.9387 40.83782, -73.93863 4...",POINT (-73.93879 40.83781)
48,3.214588,2100,11,2117853,"MULTIPOLYGON (((-73.87287 40.80269, -73.8729 4...",POINT (-73.87295 40.80269)
58,47.393352,2100,37,1018457,"MULTIPOLYGON (((-73.98237 40.74524, -73.9822 4...",POINT (-73.9823 40.74517)
68,50.966970,2100,52,1039988,"MULTIPOLYGON (((-73.96425 40.7586, -73.96421 4...",POINT (-73.96406 40.75856)
86,8.708136,2100,40,1026714,"MULTIPOLYGON (((-73.99058 40.76523, -73.99065 ...",POINT (-73.99065 40.76519)


In [149]:
### CALCULATE BUILDING DATA ######
# Convert problematic columns to numeric, with errors='coerce' to convert invalid values to NaN
numeric_columns = ['heightroof', 'groundelev']  # Added groundelev
for col in numeric_columns:
    if col in nyc_buildings.columns:
        nyc_buildings[col] = pd.to_numeric(nyc_buildings[col], errors='coerce')

# Drop rows with NaN values in critical columns
nyc_buildings = nyc_buildings.dropna(subset=['heightroof', 'groundelev'])
# Convert ground_df to GeoDataFrame
geometry = [Point(xy) for xy in zip(ground_df['Longitude'], ground_df['Latitude'])]

# Ensure both datasets use the same coordinate reference system
if nyc_buildings.crs != training_gdf.crs:
    nyc_buildings = nyc_buildings.to_crs(training_gdf.crs)

# Convert to a projected CRS for more accurate distance measurements
nyc_buildings = nyc_buildings.to_crs("EPSG:32618")
training_gdf = training_gdf.to_crs("EPSG:32618")

# Create spatial index for buildings
print("Creating spatial index...")
spatial_index = index.Index()
for idx, geom in enumerate(nyc_buildings.geometry):
    if geom is not None and geom.is_valid:
        spatial_index.insert(idx, geom.bounds)

# Add columns for metrics
training_gdf['avg_building_height'] = np.nan
training_gdf['num_buildings'] = 0
training_gdf['total_building_area'] = 0.0
training_gdf['building_density'] = 0.0  # building footprint area / buffer area
training_gdf['floor_area_ratio'] = 0.0  # gross floor area / buffer area

# Function to calculate metrics using spatial index
def calculate_metrics(point, buildings_gdf, spatial_idx, distance=100):
    if point is None:
        return 0, 0, 0, 0, 0

    # Create buffer (in meters)
    buffer = point.buffer(distance)
    buffer_area = buffer.area  # in square meters

    # Use spatial index to find potential intersections
    potential_matches_idx = list(spatial_idx.intersection(buffer.bounds))

    if len(potential_matches_idx) > 0:
        # Get the actual buildings that intersect
        potential_matches = buildings_gdf.iloc[potential_matches_idx]
        precise_matches = potential_matches[potential_matches.intersects(buffer)]

        # Updated part of the calculate_metrics function
        if len(precise_matches) > 0:
            # Calculate the bounding box area for each building
            bounding_box_areas = []
            for idx, building in precise_matches.iterrows():
                if building.geometry.is_valid:
                    bounding_box = building.geometry.envelope  # Get the bounding box
                    bounding_box_areas.append(bounding_box.area)

            # Add bounding box areas to the dataframe
            precise_matches['bounding_box_area'] = bounding_box_areas

            # Calculate metrics
            avg_height = precise_matches['heightroof'].mean()
            num_buildings = len(precise_matches)
            total_building_area = sum(bounding_box_areas)  # Use bounding box areas
            building_density = total_building_area / buffer_area

            # Use groundelev as the number of floors (assuming it represents floor count)
            floor_counts = precise_matches['groundelev'].fillna(1).clip(lower=1)
            gross_floor_area = sum(precise_matches['bounding_box_area'] * floor_counts)
            floor_area_ratio = gross_floor_area / buffer_area

            return avg_height, num_buildings, total_building_area, building_density, floor_area_ratio


    return 0, 0, 0, 0, 0  # No buildings within buffer

# Process each point with progress tracking
print("Calculating metrics...")
for idx in tqdm(range(len(training_gdf)), desc="Processing points"):
    try:
        point = training_gdf.iloc[idx].geometry
        avg_height, num_bldgs, total_area, density, far = calculate_metrics(point, nyc_buildings, spatial_index, distance=700)

        training_gdf.loc[idx, 'avg_building_height'] = avg_height
        training_gdf.loc[idx, 'num_buildings'] = num_bldgs
        training_gdf.loc[idx, 'total_building_area'] = total_area
        training_gdf.loc[idx, 'building_density'] = density
        training_gdf.loc[idx, 'floor_area_ratio'] = far
    except Exception as e:
        print(f"Error processing point {idx}: {e}")
        # Set values to NaN on error
        training_gdf.loc[idx, ['avg_building_height', 'total_building_area', 'building_density', 'floor_area_ratio']] = np.nan
        training_gdf.loc[idx, 'num_buildings'] = 0

# Convert back to original CRS for saving
training_gdf = training_gdf.to_crs("EPSG:4326")

# Save results
training_gdf.head()


Creating spatial index...
Calculating metrics...


Processing points:  81%|████████▏ | 9150/11229 [27:13<06:11,  5.60it/s]  


KeyboardInterrupt: 

In [ ]:
# Convert back to 4326
final_df=final_df.to_crs("EPSG:4326")
# Ensure training_gdf has required columns
columns_to_merge = ['Longitude','Latitude', 'avg_building_height', 'building_density', 'floor_area_ratio'] #'building_density', 'floor_area_ratio'
final_df = pd.merge(final_df, training_gdf[columns_to_merge], on=["Longitude", "Latitude"], how="inner")




In [ ]:
# final_df.to_csv("final_df.csv", index=False)  # Saves without the index column
final_df.to_file("final_allrounded.geojson", driver="GeoJSON")

# Landast 8

In [ ]:
#Extracting lst data
from rasterio.windows import Window
def map_lst_data(tiff_path, csv_path):
    # Read points from CSV
    df = pd.read_csv(csv_path)

    # Open the raster file and extract values
    with rio.open(tiff_path) as src:
        coords = list(zip(df['Longitude'], df['Latitude']))
        # lst_values = []
        band_values = {"LST": [], "NIR08": [], "RED": [], "SWIR16": [], "GREEN": []}

        for lon, lat in coords:
            try:
                row, col = src.index(lon, lat)
                window = Window(col, row, 1, 1)
                value = src.read(1, window=window)
                
                #Read values for each band
                lst_val = src.read(1, window=window)[0][0]  # Band 1: LWIR11 (LST)
                nir08_val = src.read(2, window=window)[0][0]  # Band 2: NIR08
                red_val = src.read(3, window=window)[0][0]  # Band 3: RED
                swir16_val = src.read(4, window=window)[0][0]  # Band 4: SWIR16
                green_val = src.read(5, window=window)[0][0]  # Band 5: GREEN
                
                band_values["LST"].append(float(lst_val))
                band_values["NIR08"].append(float(nir08_val))
                band_values["RED"].append(float(red_val))
                band_values["SWIR16"].append(float(swir16_val))
                band_values["GREEN"].append(float(green_val))
            except (IndexError, ValueError):
                band_values["LST"].append(None)
                band_values["NIR08"].append(None)
                band_values["RED"].append(None)
                band_values["SWIR16"].append(None)
                band_values["GREEN"].append(None)

    # Create and return output DataFrame
    return pd.DataFrame({
        'Latitude': df['Latitude'],
        'Longitude': df['Longitude'],
        'LST': band_values["LST"],
        'NIR08': band_values["NIR08"],
        'RED': band_values["RED"],
        'SWIR16': band_values["SWIR16"],
        'GREEN': band_values["GREEN"]
    })

In [ ]:
tiff_path_2 = './Features dataset/Landsat_LST.tiff'
csv_training = './Training_data_uhi_index_2025-02-18.csv'
ground_landsat = map_lst_data(tiff_path=tiff_path_2, csv_path=csv_training)

In [ ]:
# Calculate NDVI (Normalized Difference Vegetation Index) and handle division by zero by replacing infinities with NaN.
# See the Sentinel-2 sample notebook for more information about the NDVI index
ground_landsat['NDVI30'] = (ground_landsat['NIR08'] - ground_landsat['RED']) / (ground_landsat['NIR08'] + ground_landsat['RED'])
ground_landsat['NDVI30'] = ground_landsat['NDVI30'].replace([np.inf, -np.inf], np.nan)


# #EVI
# final_df['EVI'] = ((2.5*(final_df['B08'] - final_df['B04'])) / (final_df['B08'] + 6*final_df['B04'] - 7.5 * final_df['B02'] + 1))/10000
# final_df['EVI'] = final_df['EVI'].replace([np.inf, -np.inf], np.nan)


# NDBI
ground_landsat['NDBI30'] = (ground_landsat['SWIR16'] - ground_landsat['NIR08']) / (ground_landsat['SWIR16'] + ground_landsat['NIR08'])
ground_landsat['NDBI30'] = ground_landsat['NDBI30'].replace([np.inf, -np.inf], np.nan)


# NDWI
ground_landsat['NDWI30'] = (ground_landsat['GREEN'] - ground_landsat['NIR08']) / (ground_landsat['GREEN'] + ground_landsat['NIR08'])
ground_landsat['NDWI30'] = ground_landsat['NDWI30'].replace([np.inf, -np.inf], np.nan)

# #mndwi
# final_df['MNDWI'] = (final_df['GREEN'] - final_df['B11']) / (final_df['B03'] + final_df['B11'])
# final_df['MNDWI'] = final_df['MNDWI'].replace([np.inf, -np.inf], np.nan)

ground_landsat

,Latitude,Longitude,LST,NIR08,RED,SWIR16,GREEN,NDVI30,NDBI30,NDWI30
0,40.813107,-73.909167,42.345172,0.105195,0.076540,0.104453,0.068125,0.157675,-0.003542,-0.213882
1,40.813045,-73.909187,42.345172,0.105195,0.076540,0.104453,0.068125,0.157675,-0.003542,-0.213882
2,40.812978,-73.909215,41.442815,0.191820,0.070105,0.131265,0.071425,0.464694,-0.187427,-0.457350
3,40.812908,-73.909242,41.442815,0.191820,0.070105,0.131265,0.071425,0.464694,-0.187427,-0.457350
4,40.812845,-73.909257,41.152283,0.210108,0.083718,0.156537,0.085780,0.430154,-0.146109,-0.420185
...,...,...,...,...,...,...,...,...,...,...
11224,40.790333,-73.957050,34.890471,0.308640,0.048765,0.161240,0.063478,0.727116,-0.313697,-0.658831
11225,40.790308,-73.957063,34.890471,0.308640,0.048765,0.161240,0.063478,0.727116,-0.313697,-0.658831
11226,40.790270,-73.957093,34.941741,0.309328,0.050113,0.156317,0.062570,0.721163,-0.328598,-0.663509
11227,40.790253,-73.957112,34.941741,0.309328,0.050113,0.156317,0.062570,0.721163,-0.328598,-0.663509


### Landsat Emissivity

In [ ]:
# Compute Pv (Proportional Vegetation Cover)
def calculate_pv(ndvi, ndvi_min, ndvi_max):
    if ndvi < ndvi_min:
        return 0  # Bare soil
    elif ndvi > ndvi_max:
        return 1  # Dense vegetation
    else:
        return ((ndvi - ndvi_min) / (ndvi_max - ndvi_min)) ** 2

# Compute Emissivity (ε) from Pv
def calculate_emissivity(pv):
    return 0.004 * pv + 0.986

# Compute NDVI min and max from your dataset
NDVI_min = ground_landsat['NDVI30'].min()
NDVI_max = ground_landsat['NDVI30'].max()

# Apply the function to compute Pv
ground_landsat['Pv30'] = ground_landsat['NDVI30'].apply(lambda x: calculate_pv(x, NDVI_min, NDVI_max) if not np.isnan(x) else np.nan)

# Apply the function to compute Emissivity using Pv
ground_landsat['Emissivity30'] = ground_landsat['Pv30'].apply(lambda x: calculate_emissivity(x) if not np.isnan(x) else np.nan)

In [ ]:
ground_landsat

,Latitude,Longitude,LST,NIR08,RED,SWIR16,GREEN,NDVI30,NDBI30,NDWI30,Pv30,Emissivity30
0,40.813107,-73.909167,42.345172,0.105195,0.076540,0.104453,0.068125,0.157675,-0.003542,-0.213882,0.047454,0.986190
1,40.813045,-73.909187,42.345172,0.105195,0.076540,0.104453,0.068125,0.157675,-0.003542,-0.213882,0.047454,0.986190
2,40.812978,-73.909215,41.442815,0.191820,0.070105,0.131265,0.071425,0.464694,-0.187427,-0.457350,0.298668,0.987195
3,40.812908,-73.909242,41.442815,0.191820,0.070105,0.131265,0.071425,0.464694,-0.187427,-0.457350,0.298668,0.987195
4,40.812845,-73.909257,41.152283,0.210108,0.083718,0.156537,0.085780,0.430154,-0.146109,-0.420185,0.259621,0.987038
...,...,...,...,...,...,...,...,...,...,...,...,...
11224,40.790333,-73.957050,34.890471,0.308640,0.048765,0.161240,0.063478,0.727116,-0.313697,-0.658831,0.684639,0.988739
11225,40.790308,-73.957063,34.890471,0.308640,0.048765,0.161240,0.063478,0.727116,-0.313697,-0.658831,0.684639,0.988739
11226,40.790270,-73.957093,34.941741,0.309328,0.050113,0.156317,0.062570,0.721163,-0.328598,-0.663509,0.674134,0.988697
11227,40.790253,-73.957112,34.941741,0.309328,0.050113,0.156317,0.062570,0.721163,-0.328598,-0.663509,0.674134,0.988697


In [ ]:
final_df = pd.merge(final_df, ground_landsat, on=["Longitude", "Latitude"], how="inner")

# Sentinel 2

In [ ]:
#Extracting spectral data from geotiff image, allowing for buffer zone

def map_sent_data(tiff_path, csv_path, buffer_distance):
    # Read the CSV file using pandas
    df = pd.read_csv(csv_path)

    # Create points from coordinates
    geometry = [Point(lon, lat) for lon, lat in zip(df['Longitude'], df['Latitude'])]
    gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry=geometry)

    # Initialize results DataFrame with original data
    results_df = df.copy()

    with rio.open(tiff_path) as src:
        # Transform points to raster CRS
        gdf = gdf.to_crs(src.crs)

        band_name_mapping = {
            1: 'B01',
            2: 'B02',
            3: 'B03',
            4: 'B04',
            5: 'B08',
            6: 'B12',
            7: 'B09',
            8: 'B05',
            9: 'B06',
            10: 'B8A',
            11: 'B11'
        }

        # Process each point individually
        for idx, point in enumerate(tqdm(gdf.geometry, desc="Processing locations")):
            # Create buffer for this specific point
            buffered_point = point.buffer(buffer_distance)

            # Get the pixel coordinates for this specific point
            row, col = src.index(point.x, point.y)

            # Calculate window size based on buffer
            buffer_pixels = int(np.ceil(buffer_distance / src.res[0]))
            window = rio.windows.Window(
                col - buffer_pixels,
                row - buffer_pixels,
                2 * buffer_pixels + 1,
                2 * buffer_pixels + 1
            )

            # Process each band for this specific point
            for band_idx, band_name in band_name_mapping.items():
                try:
                    # Read data for this window
                    data = src.read(band_idx, window=window)

                    # Create mask for the buffer
                    shapes = [(buffered_point, 1)]
                    mask = rio.features.rasterize(
                        shapes,
                        out_shape=data.shape,
                        transform=rio.windows.transform(window, src.transform),
                        fill=0,
                        dtype='uint8'
                    )

                    # Calculate mean for masked area
                    masked_data = data[mask == 1]
                    if len(masked_data) > 0:
                        mean_value = np.mean(masked_data)
                    else:
                        # Fallback to single pixel value if no pixels in buffer
                        mean_value = src.read(band_idx, window=((row, row+1), (col, col+1)))[0][0]

                    # Assign value to specific row and band
                    results_df.at[idx, band_name] = mean_value

                except Exception as e:
                    # Fallback to single pixel value in case of any error
                    value = src.read(band_idx, window=((row, row+1), (col, col+1)))[0][0]
                    results_df.at[idx, band_name] = value

    return results_df

In [ ]:
tiff_path_3 = './Features dataset/S2_sample.tiff'
csv_training = './Training_data_uhi_index_2025-02-18.csv'
ground_sentinel = map_sent_data(tiff_path=tiff_path_3, csv_path=csv_training,buffer_distance=700)

Processing locations: 100%|██████████| 11229/11229 [19:42<00:00,  9.50it/s]


In [ ]:
# Calculate NDVI (Normalized Difference Vegetation Index) and handle division by zero by replacing infinities with NaN.
# See the Sentinel-2 sample notebook for more information about the NDVI index
ground_sentinel['NDVI10'] = (ground_sentinel['B08'] - ground_sentinel['B04']) / (ground_sentinel['B08'] + ground_sentinel['B04'])
ground_sentinel['NDVI10'] = ground_sentinel['NDVI10'].replace([np.inf, -np.inf], np.nan)


#EVI
ground_sentinel['EVI'] = ((2.5*(ground_sentinel['B08'] - ground_sentinel['B04'])) / (ground_sentinel['B08'] + 6*ground_sentinel['B04'] - 7.5 * ground_sentinel['B02'] + 1))/10000
ground_sentinel['EVI'] = ground_sentinel['EVI'].replace([np.inf, -np.inf], np.nan)


# NDBI
ground_sentinel['NDBI10'] = (ground_sentinel['B11'] - ground_sentinel['B08']) / (ground_sentinel['B11'] + ground_sentinel['B08'])
ground_sentinel['NDBI10'] = ground_sentinel['NDBI10'].replace([np.inf, -np.inf], np.nan)


# NDWI
ground_sentinel['NDWI10'] = (ground_sentinel['B03'] - ground_sentinel['B08']) / (ground_sentinel['B03'] + ground_sentinel['B08'])
ground_sentinel['NDWI10'] = ground_sentinel['NDWI10'].replace([np.inf, -np.inf], np.nan)

#mndwi
ground_sentinel['MNDWI10'] = (ground_sentinel['B03'] - ground_sentinel['B11']) / (ground_sentinel['B03'] + ground_sentinel['B11'])
ground_sentinel['MNDWI10'] = ground_sentinel['MNDWI10'].replace([np.inf, -np.inf], np.nan)


### Emissivity Sentinel 2

In [ ]:
import numpy as np

# Compute NDVI min and max from your dataset
NDVI_min = ground_sentinel['NDVI10'].min()
NDVI_max = ground_sentinel['NDVI10'].max()

# Ensure NDVI_max is reasonable (avoid division errors)
if NDVI_max == NDVI_min:
    NDVI_max = 0.5  # Fallback to default


# Apply the function to compute Pv
ground_sentinel['Pv'] = ground_sentinel['NDVI10'].apply(lambda x: calculate_pv(x, NDVI_min, NDVI_max) if not np.isnan(x) else np.nan)

# Apply the function to compute Emissivity using Pv
ground_sentinel['Emissivity'] = ground_sentinel['Pv'].apply(lambda x: calculate_emissivity(x) if not np.isnan(x) else np.nan)


### Merge Sentinel 2 to final

In [ ]:
final_df = pd.merge(final_df, ground_sentinel, on=["Longitude", "Latitude"], how="inner")

In [ ]:
final_df.columns

Index(['Longitude', 'Latitude', 'datetime_x', 'UHI Index_x', 'geometry',
       'Counties', 'Air Temp at Surface [degC]', 'Relative Humidity [percent]',
       'Avg Wind Speed [m/s]', 'Wind Direction [degrees]',
       'Solar Flux [W/m^2]', 'building_count_700m', 'building_area_700m',
       'mean_compactness_700m', 'avg_building_height', 'building_density',
       'floor_area_ratio', 'LST', 'NIR08', 'RED', 'SWIR16', 'GREEN', 'NDVI30',
       'NDBI30', 'NDWI30', 'Pv30', 'Emissivity30', 'datetime_y', 'UHI Index_y',
       'B01', 'B02', 'B03', 'B04', 'B08', 'B12', 'B09', 'B05', 'B06', 'B8A',
       'B11', 'NDVI10', 'EVI', 'NDBI10', 'NDWI10', 'MNDWI10', 'Pv',
       'Emissivity'],
      dtype='object')

In [ ]:
final_df# Rename columns
final_df.rename(columns={'datetime_x': 'datetime', 'UHI Index_x': 'UHI index'}, inplace=True)

# Drop unnecessary columns
final_df.drop(columns=['datetime_y', 'UHI Index_y'], inplace=True)

In [ ]:
final_df.columns

Index(['Longitude', 'Latitude', 'datetime', 'UHI index', 'geometry',
       'Counties', 'Air Temp at Surface [degC]', 'Relative Humidity [percent]',
       'Avg Wind Speed [m/s]', 'Wind Direction [degrees]',
       'Solar Flux [W/m^2]', 'building_count_700m', 'building_area_700m',
       'mean_compactness_700m', 'avg_building_height', 'building_density',
       'floor_area_ratio', 'LST', 'NIR08', 'RED', 'SWIR16', 'GREEN', 'NDVI30',
       'NDBI30', 'NDWI30', 'Pv30', 'Emissivity30', 'B01', 'B02', 'B03', 'B04',
       'B08', 'B12', 'B09', 'B05', 'B06', 'B8A', 'B11', 'NDVI10', 'EVI',
       'NDBI10', 'NDWI10', 'MNDWI10', 'Pv', 'Emissivity'],
      dtype='object')

In [ ]:
# final_df.to_csv("final_df.csv", index=False)  # Saves without the index column
final_df.to_file("Allrounded_finaldf.geojson", driver="GeoJSON")

## Check Null

In [ ]:

#check for nan
final_df.isna().sum()

Longitude                      0
Latitude                       0
datetime                       0
UHI index                      0
geometry                       0
Counties                       0
Air Temp at Surface [degC]     0
Relative Humidity [percent]    0
Avg Wind Speed [m/s]           0
Wind Direction [degrees]       0
Solar Flux [W/m^2]             0
building_count_700m            0
building_area_700m             0
mean_compactness_700m          0
avg_building_height            0
building_density               0
floor_area_ratio               0
LST                            0
NIR08                          0
RED                            0
SWIR16                         0
GREEN                          0
NDVI30                         0
NDBI30                         0
NDWI30                         0
Pv30                           0
Emissivity30                   0
B01                            0
B02                            0
B03                            0
B04       

# Model building

In [ ]:
#Drop the lat-lon columns
training_final = final_df.drop(columns=['Latitude', 'Longitude', 'datetime'])

#'NDVI30', 'NDBI30', 'NDWI30', 'NDVI10', 'EVI', 'NDBI10', 'NDWI10'
training_final = training_final[[ 'Relative Humidity [percent]',
       'Avg Wind Speed [m/s]', 'Wind Direction [degrees]',
       'Solar Flux [W/m^2]', 'building_count_700m', 'building_area_700m',
       'mean_compactness_700m', 'avg_building_height', 'building_density',
       'floor_area_ratio','RED','SWIR16','GREEN', 'LST', 'B01','B09', 'NDVI30', 'NDBI30', 'NDWI30', 'Emissivity30', 'UHI index']]

In [ ]:

#Split the data into features (X) and target (y), and then into training and testing sets
X = training_final.drop(columns=['UHI index']).values
y = training_final['UHI index'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,random_state=42)

In [ ]:
# Scale the training and test data using standardscaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#Train the Random Forest model on the training data
model = RandomForestRegressor(n_estimators=100, random_state=142)
model.fit(X_train,y_train)

RandomForestRegressor(random_state=142)

In [ ]:
#Make predictions on the training data
insample_predictions = model.predict(X_train)
#calculate R-squared score for in-sample predictions
Y_train = y_train.tolist()
r2_score(Y_train, insample_predictions)

0.9934008589099509

In [ ]:

#Make predictions on the test data
outsample_predictions = model.predict(X_test)

#calculate R-squared score for out-sample predictions
Y_test = y_test.tolist()
r2_score(Y_test, outsample_predictions)

0.9552979232721228

<!-- Selected feature:

'Air Temp at Surface [degC]', 'Relative Humidity [percent]', 'Wind Direction [degrees]','building_count_700m','building_area_700m','mean_compactness_700m','avg_building_height','floor_area_ratio','LST','RED','NDVI30','NDBI30','NDWI30','Emissivity30','B01','B02','B03','B04','B12','B05' -->